<a href="https://colab.research.google.com/github/limshaocong/analyticsEdge/blob/main/Zero_shot_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install transformers==3.1.0

     |████████████████████████████████| 884 kB 8.4 MB/s 
     |████████████████████████████████| 3.0 MB 34.2 MB/s 
     |████████████████████████████████| 895 kB 63.1 MB/s 
     |████████████████████████████████| 1.2 MB 55.9 MB/s 


In [4]:
from transformers import pipeline
import pandas as pd

In [5]:
classifier = pipeline("zero-shot-classification", # transfer learning
                      model = "joeddav/xlm-roberta-large-xnli", # multi-lingual model
                      device = 0) # to utilize GPU

Downloading:   0%|          | 0.00/734 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

In [6]:
import re
from datetime import datetime
import os.path
from os import path
import requests

In [7]:
def analyse(ticker):
  
  file_path = path + ticker + str("_tweets_2020_cleaned.csv")
  
  df = pd.read_csv(file_path)
  
  df['Text_clean'] = df['Text_clean'].astype(str)

  texts = df["Text_clean"]

  chunks = [texts[x : x + 100] for x in range(0, len(texts), 100)] # split into chunks

  results_list = []

  for idx in range(len(chunks)):

    cp1 = datetime.now()
    
    chunk = chunks[idx].tolist()

    candidate_labels = ["positive", "negative", "neutral"]

    hypothesis_template = "The sentiment of this tweet is {}."

    #dicts = classifier(text, candidate_labels, hypothesis_template = hypothesis_template)
    #result = dicts["labels"][0]
    #results_list.append(result)
    
    list_results_dicts = classifier(chunk, candidate_labels, hypothesis_template = hypothesis_template)

    if type(list_results_dicts) != dict:
      new_list = [i["labels"][0] for i in list_results_dicts] # not robust when chunk size == 1
      results_list.extend(new_list)
    
    else:
      new_list = list_results_dicts["labels"][0]
      results_list.append(new_list)
    
    #results_list.extend(new_list)

    cp2 = datetime.now()

    print(str(idx + 1) + str("/") + str(len(chunks)) + str(" --- ") + str(cp2 - cp1))

  return results_list

In [24]:
ticker_path = "https://raw.githubusercontent.com/limshaocong/analyticsEdge/main/TwitterScraper/Tickers_reduced_full.csv"
tickers = pd.read_csv(ticker_path)
tickers_list = tickers["Tickers"].tolist()
tickers_list = tickers_list[17:]

path = "https://raw.githubusercontent.com/limshaocong/analyticsEdge/main//Datasets/Tweet_Analysis/"

In [26]:
skipped = ["AMZN", "BA"]
last = ["BK"]
tickers_list[:5]

['BK', 'BRK.B', 'BSX', 'C', 'CAT']

In [23]:
for ticker in tickers_list:
  
  file_path = path + ticker + str("_tweets_2020_cleaned.csv")
  
  response = requests.get(file_path)

  if response.status_code < 400:
 
    print(ticker + str(" is present"))

    file_path = path + ticker + str("_tweets_2020_cleaned.csv")

    print(file_path)
  
    df = pd.read_csv(file_path)

    results_list = analyse(ticker)

    df["Sentiment"] = results_list

    newfilename = ticker + str("_tweets_2020_tagged.csv")

    df.to_csv(newfilename, index = False)
  
  else:

    print(ticker + str(" is missing"))

    continue

BAC is present
https://raw.githubusercontent.com/limshaocong/analyticsEdge/main//Datasets/Tweet_Analysis/BAC_tweets_2020_cleaned.csv
1/604 --- 0:00:03.315333
2/604 --- 0:00:03.236088
3/604 --- 0:00:03.238348
4/604 --- 0:00:03.136857
5/604 --- 0:00:02.995312
6/604 --- 0:00:03.229948
7/604 --- 0:00:03.227996
8/604 --- 0:00:02.551017
9/604 --- 0:00:02.966958
10/604 --- 0:00:02.981830
11/604 --- 0:00:03.218719
12/604 --- 0:00:03.123824
13/604 --- 0:00:02.908769
14/604 --- 0:00:03.113807
15/604 --- 0:00:02.964150
16/604 --- 0:00:02.883898
17/604 --- 0:00:03.053421
18/604 --- 0:00:02.776806
19/604 --- 0:00:02.479230
20/604 --- 0:00:03.049055
21/604 --- 0:00:02.932991
22/604 --- 0:00:02.981432
23/604 --- 0:00:02.894365
24/604 --- 0:00:03.123294
25/604 --- 0:00:03.122712
26/604 --- 0:00:02.948129
27/604 --- 0:00:03.053027
28/604 --- 0:00:02.976129
29/604 --- 0:00:03.051235
30/604 --- 0:00:03.117105
31/604 --- 0:00:03.048639
32/604 --- 0:00:03.050987
33/604 --- 0:00:03.051026
34/604 --- 0:00:03

KeyboardInterrupt: ignored